 Tutaj zaimportuj potrzebne biblioteki

In [1]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import dash
import pandas as pd
import plotly.express as px
import threading
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import dash_table


C:\Users\admin\AppData\Local\Temp\ipykernel_6184\1077277139.py:15: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table


 ## Konfiguracja połączenia
 Tutaj uzupełnij konfigurację połączenia

In [4]:
username = 'postgres'
password = 'password'

host = 'localhost'
database = 'airlines'
port = 5432

 Tutaj zdefiniuj zmienną `con` oraz `cursor`

In [5]:
conn = psycopg2.connect(database="airlines", user="postgres", password="password", host="localhost", port="5432")

cur = conn.cursor()


In [6]:
# Utworzenie widoku
with conn.cursor() as cur:
    cur.execute("""
        CREATE OR REPLACE VIEW reporting.top_reliability_roads AS
        SELECT f.origin_airport_id,
            al1.name AS origin_airport_name,
            f.dest_airport_id,
            al2.name AS dest_airport_name,
            f.year,
            COUNT(*) AS cnt,
            ROUND(AVG(
                CASE
                    WHEN f.dep_delay_new > 0::double precision THEN 1
                    ELSE 0
                END), 2) AS reliability,
            RANK() OVER (ORDER BY AVG(
                CASE
                    WHEN f.dep_delay_new > 0::double precision THEN 1
                    ELSE 0
                END) DESC) AS nb
        FROM flight f
        JOIN airport_list al1 ON f.origin_airport_id = al1.origin_airport_id
        JOIN airport_list al2 ON f.dest_airport_id = al2.origin_airport_id
        GROUP BY f.origin_airport_id, al1.name, f.dest_airport_id, al2.name, f.year
        HAVING COUNT(*) > 10000
        ORDER BY COUNT(*) DESC;
    """)

top_routes_df = pd.read_sql_query("""
SELECT origin_airport_name, dest_airport_name,
            year,
            reliability,
            nb
        FROM reporting.top_reliability_roads
        ORDER BY nb ASC
        LIMIT 10;
    """, conn)

top_routes_df

C:\Users\admin\AppData\Local\Temp\ipykernel_16956\1670977705.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  top_routes_df = pd.read_sql_query("""


,origin_airport_name,dest_airport_name,year,reliability,nb
0,"CHICAGO OHARE INTERNATIONAL AIRPORT, IL US","LAGUARDIA AIRPORT, NY US",2019,0.40,1
1,"LOS ANGELES INTERNATIONAL AIRPORT, CA US","LAGUARDIA AIRPORT, NY US",2019,0.38,2
2,"LOS ANGELES INTERNATIONAL AIRPORT, CA US","MCCARRAN INTERNATIONAL AIRPORT, NV US",2019,0.37,3
3,"MCCARRAN INTERNATIONAL AIRPORT, NV US","LOS ANGELES INTERNATIONAL AIRPORT, CA US",2019,0.36,4
4,"LOS ANGELES INTERNATIONAL AIRPORT, CA US","SAN FRANCISCO INTERNATIONAL AIRPORT, CA US",2019,0.35,5
5,"LAGUARDIA AIRPORT, NY US","CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",2019,0.35,6
6,"BOSTON, MA US","LAGUARDIA AIRPORT, NY US",2019,0.34,7
7,"LAGUARDIA AIRPORT, NY US","BOSTON, MA US",2019,0.34,8
8,"SAN FRANCISCO INTERNATIONAL AIRPORT, CA US","LOS ANGELES INTERNATIONAL AIRPORT, CA US",2019,0.33,9
9,"LAGUARDIA AIRPORT, NY US","LOS ANGELES INTERNATIONAL AIRPORT, CA US",2019,0.27,10


 Tutaj dokonaj aktualizacji formatu liczbowego dla kolumny `reliability`

In [7]:
top_routes_df ['reliability'] = top_routes_df ['reliability'].apply(lambda x: '{:.2%}'.format(x))
top_routes_df 

,origin_airport_name,dest_airport_name,year,reliability,nb
0,"CHICAGO OHARE INTERNATIONAL AIRPORT, IL US","LAGUARDIA AIRPORT, NY US",2019,40.00%,1
1,"LOS ANGELES INTERNATIONAL AIRPORT, CA US","LAGUARDIA AIRPORT, NY US",2019,38.00%,2
2,"LOS ANGELES INTERNATIONAL AIRPORT, CA US","MCCARRAN INTERNATIONAL AIRPORT, NV US",2019,37.00%,3
3,"MCCARRAN INTERNATIONAL AIRPORT, NV US","LOS ANGELES INTERNATIONAL AIRPORT, CA US",2019,36.00%,4
4,"LOS ANGELES INTERNATIONAL AIRPORT, CA US","SAN FRANCISCO INTERNATIONAL AIRPORT, CA US",2019,35.00%,5
5,"LAGUARDIA AIRPORT, NY US","CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",2019,35.00%,6
6,"BOSTON, MA US","LAGUARDIA AIRPORT, NY US",2019,34.00%,7
7,"LAGUARDIA AIRPORT, NY US","BOSTON, MA US",2019,34.00%,8
8,"SAN FRANCISCO INTERNATIONAL AIRPORT, CA US","LOS ANGELES INTERNATIONAL AIRPORT, CA US",2019,33.00%,9
9,"LAGUARDIA AIRPORT, NY US","LOS ANGELES INTERNATIONAL AIRPORT, CA US",2019,27.00%,10


 Tutaj odpowiednio dokonaj zmian nazewnictwa kolumn

In [8]:
top_routes_df = top_routes_df.rename(columns={'origin_airport_name': 'Origin Name','dest_airport_name': 'Destination Name', 'year': 'Year', 'reliability': 'Reliability', 'nb':'Rank'})
top_routes_df

,Origin Name,Destination Name,Year,Reliability,Rank
0,"CHICAGO OHARE INTERNATIONAL AIRPORT, IL US","LAGUARDIA AIRPORT, NY US",2019,40.00%,1
1,"LOS ANGELES INTERNATIONAL AIRPORT, CA US","LAGUARDIA AIRPORT, NY US",2019,38.00%,2
2,"LOS ANGELES INTERNATIONAL AIRPORT, CA US","MCCARRAN INTERNATIONAL AIRPORT, NV US",2019,37.00%,3
3,"MCCARRAN INTERNATIONAL AIRPORT, NV US","LOS ANGELES INTERNATIONAL AIRPORT, CA US",2019,36.00%,4
4,"LOS ANGELES INTERNATIONAL AIRPORT, CA US","SAN FRANCISCO INTERNATIONAL AIRPORT, CA US",2019,35.00%,5
5,"LAGUARDIA AIRPORT, NY US","CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",2019,35.00%,6
6,"BOSTON, MA US","LAGUARDIA AIRPORT, NY US",2019,34.00%,7
7,"LAGUARDIA AIRPORT, NY US","BOSTON, MA US",2019,34.00%,8
8,"SAN FRANCISCO INTERNATIONAL AIRPORT, CA US","LOS ANGELES INTERNATIONAL AIRPORT, CA US",2019,33.00%,9
9,"LAGUARDIA AIRPORT, NY US","LOS ANGELES INTERNATIONAL AIRPORT, CA US",2019,27.00%,10


 Tutaj pobierz z widoku `reporting.year_to_year_comparision` dane do ramki `yoy_comparision_df`

In [9]:
yoy_comparision_df = pd.read_sql('SELECT * FROM reporting.year_to_year_comparision', conn)
yoy_comparision_df

C:\Users\admin\AppData\Local\Temp\ipykernel_16956\2052281046.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  yoy_comparision_df = pd.read_sql('SELECT * FROM reporting.year_to_year_comparision', conn)


,year,month,flights_amount,reliability
0,2019,1,89455,0.34
1,2019,2,81574,0.39
2,2019,3,93862,0.35
3,2019,4,91434,0.34
4,2019,5,96097,0.38
5,2019,6,94948,0.43
6,2019,7,98042,0.39
7,2019,8,100496,0.38
8,2019,9,92090,0.30
9,2019,10,96566,0.33


 Tutaj odpowiednio przekształć ramkę, do formy oczekiwanej przez wykres słupkowy. Wyniki zapisz do zmiennej `yoy_comparision_to_plot_df`

In [10]:
yoy_comparision_to_plot_df = yoy_comparision_df.pivot(index='month', columns='year', values='reliability')
yoy_comparision_to_plot_df

year,2019,2020
month,,
1,0.34,0.28
2,0.39,0.28
3,0.35,0.16
4,0.34,NaN
5,0.38,NaN
6,0.43,NaN
7,0.39,NaN
8,0.38,NaN
9,0.30,NaN


 Tutaj stwórz odpowiedni wykres, zapisz do na później pod nazwą `yoy_comparision_fig`. Pamiętaj o dobrych praktykach - niech wykres posiada wszystkie opisy oraz opisowe etykiety osi.

In [11]:
month_labels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']


yoy_comparision_fig = px.bar(yoy_comparision_to_plot_df, barmode='group', labels={'value': 'Reliability', 'month': 'Month', 'year': 'Year'}, text=month_labels)
yoy_comparision_fig.update_layout(title='Year to Year Comparision (2019 vs 2020)')
# yoy_comparision_fig.update_traces(textposition='outside')

yoy_comparision_fig

 Tutaj pobierz z widoku `reporting.day_to_day_comparision` dane do ramki `day_to_day_comparision_df`

In [12]:
day_to_day_comparision_df = pd.read_sql('SELECT * FROM reporting.day_to_day_comparision', conn)
day_to_day_comparision_df

C:\Users\admin\AppData\Local\Temp\ipykernel_16956\1722328444.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,year,day_of_week,flights_amount
0,2019,1,167761
1,2019,2,165478
2,2019,3,163889
3,2019,4,166154
4,2019,5,167397
5,2019,6,129485
6,2019,7,155663
7,2020,1,40983
8,2020,2,39807
9,2020,3,39581


 Tutaj odpowiednio przekształć ramkę, do formy oczekiwanej przez wykres słupkowy. Wyniki zapisz do zmiennej `day_to_day_comparision_to_plot_df`

In [13]:
day_to_day_comparision_to_plot_df = day_to_day_comparision_df.pivot(index='day_of_week', columns='year', values='flights_amount')
day_to_day_comparision_to_plot_df

year,2019,2020
day_of_week,,
1,167761,40983
2,165478,39807
3,163889,39581
4,166154,41086
5,167397,41058
6,129485,30545
7,155663,37233


 Tutaj stwórz odpowiedni wykres, zapisz do na później pod nazwą `day_to_day_comparision_fig`. Pamiętaj o dobrych praktykach - niech wykres posiada wszystkie opisy oraz opisowe etykiety osi.

In [14]:
day_to_day_comparision_fig = px.bar(day_to_day_comparision_to_plot_df, barmode='group', labels={'value': 'Reliability', 'day_of_week': 'Day of week', 'year': 'Year'})
day_to_day_comparision_fig.update_layout(title='Day-to-Day Comparison 2019 - 2020')
day_to_day_comparision_fig

 # Stworzenie szeregu czasowego
 W tym miejscu stworzymy wykres w formie szeregu czasowego, który zostanie umieszcony w raporcie. Wykres powinien przedstawiać dane w formie szeregu czasowego (dzień po dniu) dla lat 2019 oraz 2020. Dla możliwości identyfikacji poszczególnych lat, zostaliśmy dodatkowo poproszeni o nadanie innego koloru dla roku 2019 oraz 2020.

 > Pamiętaj o stworzeniu wykresu zgodnie z dobrymi praktykami.

 W tym miejscu pobierz dane do ramki `day_by_day_reliability_df`, z widoku `reporting.day_by_day_reliability`.

In [15]:
day_by_day_reliability_df = pd.read_sql('SELECT * FROM reporting.day_by_day_reliability', conn)
day_by_day_reliability_df

C:\Users\admin\AppData\Local\Temp\ipykernel_16956\3834256501.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,date,reliability
0,2019-01-01,0.40
1,2019-01-02,0.44
2,2019-01-03,0.35
3,2019-01-04,0.29
4,2019-01-05,0.37
...,...,...
451,2020-03-27,0.05
452,2020-03-28,0.07
453,2020-03-29,0.05
454,2020-03-30,0.05


 Tutaj stwórz wykres liniowy na podstawie pobranych danych. Wynik zapisz do zmiennej `day_by_day_reliability_fig`.

In [16]:
# day_by_day_reliability_fig = px.line(day_by_day_reliability_df, x='date', y='reliability')
# day_by_day_reliability_fig.show()
day_by_day_reliability_df['date'] = pd.to_datetime(day_by_day_reliability_df['date'])

day_by_day_reliability_fig = px.line(day_by_day_reliability_df, x='date', y='reliability', color=day_by_day_reliability_df['date'].apply(lambda x: '2020' if x.year == 2020 else '2019'))
day_by_day_reliability_fig.update_layout(title='Day-to-Day Reliability 2019 - 2020')
day_by_day_reliability_fig.show()

 # Stworzenie layoutów poszczególnych stron

 Tutaj przygotuj komponent `Dash` `DataTable`, zapisując wynik do zmiennej `top_routes_table`

In [36]:
top_routes_table = dash_table.DataTable(
    id='top-routes-table',
    columns=[{"name": i, "id": i} for i in top_routes_df.columns],
    data=top_routes_df.to_dict('records')
)
top_routes_table


DataTable(data=[{'Origin Name': 'CHICAGO OHARE INTERNATIONAL AIRPORT, IL US', 'Destination Name': 'LAGUARDIA AIRPORT, NY US', 'Year': 2019, 'Reliability': '40.00%', 'Rank': 1}, {'Origin Name': 'LOS ANGELES INTERNATIONAL AIRPORT, CA US', 'Destination Name': 'LAGUARDIA AIRPORT, NY US', 'Year': 2019, 'Reliability': '38.00%', 'Rank': 2}, {'Origin Name': 'LOS ANGELES INTERNATIONAL AIRPORT, CA US', 'Destination Name': 'MCCARRAN INTERNATIONAL AIRPORT, NV US', 'Year': 2019, 'Reliability': '37.00%', 'Rank': 3}, {'Origin Name': 'MCCARRAN INTERNATIONAL AIRPORT, NV US', 'Destination Name': 'LOS ANGELES INTERNATIONAL AIRPORT, CA US', 'Year': 2019, 'Reliability': '36.00%', 'Rank': 4}, {'Origin Name': 'LOS ANGELES INTERNATIONAL AIRPORT, CA US', 'Destination Name': 'SAN FRANCISCO INTERNATIONAL AIRPORT, CA US', 'Year': 2019, 'Reliability': '35.00%', 'Rank': 5}, {'Origin Name': 'LAGUARDIA AIRPORT, NY US', 'Destination Name': 'CHICAGO OHARE INTERNATIONAL AIRPORT, IL US', 'Year': 2019, 'Reliability': '35.

 Tutaj przygotuj nagłówek dla tabeli, zapisz go pod nazwą `top_routes_page_title`

In [37]:
top_routes_page_title = html.H3("Top 10 Routes")

 Tutaj przygotuj layout dla widoku `TOP 10 routes`. Konfigurację zapisz do zmiennej `top_routes_layout`

In [39]:
top_routes_page_title = html.H3(
    children='TOP 10 reliability routes in 2019 and 2020',
    style={
        'font-family': 'verdana',
        'color': '#4444'
    }
)

top_routes_table = dash_table.DataTable(
    id='top-routes-table',
    columns=[{'name': i, 'id': i} for i in top_routes_df.columns],
    data=top_routes_df.to_dict('records')
)

top_routes_layout = html.Div(
    children=[
        top_routes_page_title,
        top_routes_table
    ],
    style={
        'margin': '50px'
    }
)


 Tutaj opakuj `yoy_comparision_fig` w odpowiedni komponent `Dash`. Wynik zapisz do zmiennej `yoy_comparision_component`.

In [40]:
yoy_comparison_component = dcc.Graph(figure=yoy_comparision_fig)

 Tutaj opakuj `day_to_day_comparision_fig` w odpowiedni komponent `Dash`. Wynik zapisz do zmiennej `day_to_day_comparision_component`.

In [41]:
day_to_day_comparision_component = dcc.Graph(figure=day_to_day_comparision_fig)

 Tutaj stwórz odpowiedni `layout` dla widoku `comparision`. Wynik zapisz do zmiennej `comparision_layout`.

In [42]:
comparision_layout = html.Div([
    html.H3('Comparison between 2019 and 2020'),
    html.Div([
        html.Div([
            html.H4('Year over year comparison'),
            yoy_comparision_fig
        ], className='six columns'),
        html.Div([
            html.H4('Day to day comparison'),
            day_to_day_comparision_component
        ], className='six columns')
    ], className='row')
])


 Tutaj stwórz odpowiedni `layout` dla widoku `day by day`. Wynik zapisz do zmiennej `day_by_day_layout`.

In [43]:
day_by_day_layout = html.Div([
    html.H1('Day by Day Reporting'),
    dcc.Graph(id='day_to_day_comparision', figure=day_by_day_reliability_fig),
])

 Tutaj przygotuj layout aplikacji

In [44]:
app = dash.Dash(__name__)


app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div([
        dcc.Link('TOP report ', href='/top_report'),
        dcc.Link('Comparision ', href='/comparision_reporting'),
        dcc.Link('Day by day reporting ', href='/day_by_day_reporting'),
    ], className='row'),
    html.Div(id='page-content')
])


 Tutaj stwórz procedurę odpowiadającą za nawigację pomiędzy poszczególnymi stronami

In [45]:

@app.callback(
    Output("page-content", "children"),
    Input("url", "pathname"),
)
def display_page(pathname):
    if pathname == "/comparision_reporting":
        return comparision_layout
    elif pathname == "/day_by_day_reporting":
        return day_by_day_layout
    else:
        return top_routes_layout

 Tutaj uruchom aplikację

In [ ]:
app.run_server(port=8070)